In [1]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from spn.structure.leaves.parametric.Parametric import Categorical, Gaussian

import pandas as pd
import numpy as np

bunch = fetch_openml('mushroom')
print ("%d rows" % len(bunch.frame))
bunch.frame.head()

8124 rows


,cap-shape,cap-surface,cap-color,bruises%3F,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,class
0,x,s,n,t,p,f,c,n,k,e,...,w,w,p,w,o,p,k,s,u,p
1,x,s,y,t,a,f,c,b,k,e,...,w,w,p,w,o,p,n,n,g,e
2,b,s,w,t,l,f,c,b,n,e,...,w,w,p,w,o,p,n,n,m,e
3,x,y,w,t,p,f,c,n,n,e,...,w,w,p,w,o,p,k,s,u,p
4,x,s,g,f,n,f,w,b,k,t,...,w,w,p,w,o,e,n,a,g,e


In [2]:
frame = bunch.frame.dropna()
names = frame.columns.tolist()
cols = []
value_text = []
for name in names:
    values, text = pd.factorize(frame[name], sort = True)    
    value_text.append(text.tolist())
    cols.append(values)

data = np.stack(cols, axis = 1)
ptypes = [Categorical for _ in names]

categorical = [True]*len(names)

train, test = train_test_split(data, stratify = frame['class'], random_state = 0)
print ("%d in train, %d in test" % (len(train), len(test)))

4233 in train, 1411 in test


In [3]:
from spn.algorithms.LearningWrappers import learn_parametric
from spn.structure.Base import Context

net = learn_parametric(train, 
                       ds_context = Context(parametric_types=ptypes).add_domains(train), 
                       rows = "gmm", ohe = True,
                       min_instances_slice = len(train) / 100)

/home/saurabh/miniconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
/home/saurabh/miniconda3/lib/python3.7/site-packages/sklearn/cross_decomposition/_pls.py:98: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
/home/saurabh/miniconda3/lib/python3.7/site-packages/sklearn/cross_decomposition/_pls.py:98: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)


In [15]:
import csi2
from spn.structure.Base import get_nodes_by_type, Product

def format_condition(condition):
    try:
        a, sign, b = condition.split(" ")
    except ValueError as e:
        raise ValueError("Some nodes don't have conditions. reduce min_impurity_decrease")
    
    i = names.index(a)
    v = value_text[i][int(b)]
    
    return "%s %s %s" % (a, sign, v)

print ('#product nodes = ', len(get_nodes_by_type(net, (Product))))
net.n_instances = len(train)
csi2.annotate_spn(net, names, categorical, 
             min_impurity_decrease = 0.1, max_depth = 2)

rules = csi2.context_specific_independences(net, instance_threshold = 0)
csis = []
ac = []
cc = []
for i, rule in enumerate(rules):
    antecedent, consequent, *scores = rule
    A = csi2.format_antecedent(antecedent, format_condition)
    C = csi2.format_consequent(consequent)
    a_count = csi2.antecedent_count(A)
    c_count = csi2.consequent_count(C)
    ac.append(a_count)
    cc.append(c_count)
    csis.append ("{%s} => {%s} | %.2f, %.2f, %d | %d %d" % (A, C, *scores, a_count, c_count))
print ("%d, %.2f, %.2f" % (len(csis), np.mean(ac), np.mean(cc)))

rules = csi2.context_specific_independences(net, instance_threshold = len(train) / 20,
                                           precision_threshold = 0.7, recall_threshold = 0.7)
csis = []
ac = []
cc = []
for i, rule in enumerate(rules):
    antecedent, consequent, *scores = rule
    A = csi2.format_antecedent(antecedent, format_condition)
    C = csi2.format_consequent(consequent)
    a_count = csi2.antecedent_count(A)
    c_count = csi2.consequent_count(C)
    ac.append(a_count)
    cc.append(c_count)
    csis.append ("{%s} => {%s} | %.2f, %.2f, %d | %d %d" % (A, C, *scores, a_count, c_count))
print ("%d, %.2f, %.2f" % (len(csis), np.mean(ac), np.mean(cc)))

#product nodes =  39
39, 5.90, 8.54
14, 4.79, 7.93


In [13]:
csis

['{} => {(veil-type), (cap-shape,cap-surface,cap-color,bruises%3F,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-color,ring-number,ring-type,spore-print-color,population,habitat,class)} | 1.00, 1.00, 4233 | 1 2',
 '{(habitat != d) | ((habitat == d) & (class != e))} => {(gill-attachment), (veil-color), (ring-number), (cap-shape,cap-surface,cap-color,bruises%3F,odor,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,ring-type,spore-print-color,population,habitat,class)} | 0.94, 1.00, 2708 | 3 4',
 '{[(habitat == d) & (class == e)] & [stalk-shape != e]} => {(bruises%3F), (gill-attachment), (stalk-shape), (stalk-root), (stalk-surface-above-ring), (stalk-surface-below-ring), (veil-color), (ring-number), (ring-type), (habitat), (class), (cap-shape), (cap-surface

In [16]:
from spn.algorithms.Inference import log_likelihood

print ("test log-likelihood = %.2f" % np.mean(log_likelihood(net, test)))

test log-likelihood = -8.98
